# Imaginary time evolution

$
\require{physics}
\def\bm{\boldsymbol}
$

In [ ]:
from copy import deepcopy

import numpy as np
from scipy.linalg import expm
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
%matplotlib inline

from ph121c_lxvm import tensor, basis, models

## What we are doing

In this assignment there will be simulations of the dynamics of quantum systems
with local Hamiltonians in the matrix product state (MPS) representation.
This will be via the Time Evolving Block Decimation (TEBD) algorithm.

If you're like me and you're learning this for the first time, I looked for help
on the topic and have created this brief collection of learning materials:

- [Roger Penrose tensor diagram notation (1971)](https://www.mscs.dal.ca/%7Eselinger/papers/graphical-bib/public/Penrose-applications-of-negative-dimensional-tensors.pdfhttps://www.mscs.dal.ca/%7Eselinger/papers/graphical-bib/public/Penrose-applications-of-negative-dimensional-tensors.pdf)
- [Vidal's exposition of TEBD (2004)](https://arxiv.org/abs/quant-ph/0310089)
- [Tensor Network TEBD](https://tensornetwork.org/mps/algorithms/timeevo/tebd.html)
- [TeNPy (see `literature` for more)](https://tenpy.github.io/index.html)
- [Help with tensor networks](https://www.tensors.net/)
- [White's exposition of DMRG (1992)](https://doi.org/10.1103/PhysRevLett.69.2863)

## Overview

In TEBD, we are interested in applying a propagator $U(t)$ to evolve a quantum
state. In this section, we will use imaginary time evolution of the Hamiltonian:
$$ U(\tau) = e^{H \tau}, $$
but next section we will consider real time evolution:
$$ U(t) = e^{-i H t}. $$
We will use the following TFIM Hamiltonian with open boundary conditions
parametrized by $h^x, h^z$:
$$
    H = 
        -\sum_{i=1}^{L-1} \sigma_i^z \sigma_{i+1}^z
        -h^x \sum_{i=1}^L \sigma_i^x 
        -h^z \sum_{i=1}^L \sigma_i^z 
    .
$$
Because the Hamiltonian is 2-local, we can group its terms:
\begin{align}
    H &=
        -\left( \sum_{i=1}^L h^x \sigma_i^x + h^z \sigma_i^z \right)
        - \left( \sum_{i=1}^{L // 2} \sigma_{2i - 1}^z \sigma_{2i}^z \right)
        - \left( \sum_{i=1}^{L // 2-1+L\%2} \sigma_{2i}^z \sigma_{2i+1}^z \right)
    \\\\
        &= H_1 + H_2^\text{even} + H_2^\text{odd}
        .
\end{align}
Now all of the terms in each group commute, but the groups themselves may not.
The Zassenhaus formula [(proven here)](https://doi.org/10.1002/cpa.3160070404)
allows a separation of $U(t)$ into a product of matrix exponentials of these
local terms, grouped in powers of $t$.
The formula tells us that the lowest order of $t$ in the exponent is given by:
$$
    U(\tau) =
    e^{H_1 \tau} e^{H_2^\text{even} \tau} e^{H_2^\text{odd} \tau}
    e^{\mathcal O (\tau^2)}
    .
$$
Note: the Zassenhaus formula looks like the Baker-Campbell-Hausdorff formula,
but they group terms differently. The former groups terms in powers of $t$, and
thus lends itself to perturbative approximations.

A corollary of the Zassenhaus formula is the Lie product formula:
$$
    e^{A + B} = \lim_{N \to \infty} \left( e^{A/N} e^{B/N} \right)^N
    .
$$
In fact, this result was known as early as 1893 by Sophus Lie, the namesake of
Lie algebras! Their work is published in ISBN 0828402329 and on the
[web](https://archive.org/details/theoriedertrans00liegoog?ref=ol&view=theater).
Thus for some finite $N$, we will are in a good position to make the 
[Suzuki](https://aip.scitation.org/doi/abs/10.1063/1.529425)-[Trotter
](https://doi.org/10.1090/S0002-9939-1959-0108732-6) decomposition
which provides an approximation of the time-evolution operator over a discrete
number of time steps:
$$
    U(\tau) \approx
        \left(
        e^{H_1 \tau/N} e^{H_2^\text{even} \tau/N} e^{H_2^\text{odd} \tau/N}
        \right)^N
    .
$$
This provides yet another example of how mathematicians have explored areas
relevant to physics more than a century before their emergence.

In practice, **this is how we will simulate time evolution of quantum systems**
on both classical computers (using the MPS representation) _and_ on quantum
computers by applying the gates on qudits in the same fashion as on the tensor
network. For higher-order Suzuki-Trotter decompositions,
[read this (cited by Vitali)](https://arxiv.org/abs/quant-ph/9809009).

## Matrix calculus

I haven't given you enough details yet to apply this to a tensor network!
I haven't exponentiated matrices that can be written as Kronecker products.

In general, matrix exponentials can be calculated in terms of diagonalizing a
matrix and exponentiating its eigenvalues, or by a power-series representation.
For a review of matrix exponentials, see these [course notes written by my summer
research mentor](https://web.mit.edu/18.06/www/Spring17/Matrix-Exponentials.pdf),
or this [paper on many ways of calculating the matrix exponential
](http://www.cs.cornell.edu/cvResearchPDF/19ways+.pdf).

My questions about Kronecker products and matrix exponentials center around
whether the operations commute: for 1-site operators, 2-site operators?
Let's begin

In [ ]:
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sz = np.diag([1, -1])

### Two sites
For the diagonal matrix $\sigma^z$ we shall see that the matrix exponential and
Kronecker product do not commute for a 2-site operator:

In [ ]:
expm(np.kron(sz, sz))

In [ ]:
np.kron(expm(sz), expm(sz))

Not the same! This means we actually have to exponentiate the matrix, which for
a diagonal matrix is equivalent to exponentiating the diagonal:

In [ ]:
np.exp(np.diag(np.kron(sz, sz)))

The consequence for MPS is that this is not possible to represent as a one-site
operator, so we will have to contract a virtual index before applying this
transformation directly. (After, we will again do an SVD to return to MPS form).

We _could_ represent the matrix product operator as acting on individual sites
if we take an SVD of it and disaggregating the physical indicies by introducing
a virtual index. This option might not be viable for diagonal matrices with
large coefficients (maybe better for random unitaries):

In [ ]:
np.linalg.svd(expm(np.kron(sz, sz)))

That is, if we can write the elements of a 2-site operator $U(t)$ as
$U_{\sigma'_i \sigma'_{i+1}}^{\sigma_i \sigma_{i+1}}$, then we should reshape
the matrix so that $U_{\sigma'_i \sigma_i}^{\sigma'_{i+1} \sigma_{i+1}}$
groups the physical indices by the site. Then we should do an SVD on this matrix
which will introduce a virtual index $\alpha$, leading to:
$$
    U_{\sigma'_i \sigma_i}^{\sigma'_{i+1} \sigma_{i+1}}
        = \sum_\alpha U_{\sigma'_i \sigma_i}^\alpha
        S_\alpha^\alpha (V^\dagger)_{\alpha}^{\sigma'_{i+1} \sigma_{i+1}}
    .
$$
We should then reshape
$U_{\sigma'_i \sigma_i}^\alpha \to U_{\sigma'_i}^{\sigma_i \alpha}$ and
$(V^\dagger)_{\alpha}^{\sigma'_{i+1} \sigma_{i+1}}
\to (V^\dagger)_{\alpha \sigma'_{i+1}}^{\sigma_{i+1}}$
so that we can apply these operators to the physical indices as a matrix
multiplication. This opens the door to matrix product operators, which have very
similar properties as MPS, except for the additional physical index.

### One site

#### Diagonal matrix

We shall see that for a 1-site operator, the matrix exponential commutes with
a Kronecker product by the identity:

In [ ]:
expm(np.kron(np.eye(2), sz))

In [ ]:
np.kron(np.eye(2), expm(sz))

#### Non-diagonal matrix

For the off-diagonal matrix $\sigma^x$, the operations still commute across
Kronecker products with the identity:

In [ ]:
expm(np.kron(np.eye(2), sx))

In [ ]:
np.kron(np.eye(2), expm(sx))

All in all, this means we can time-evolve local operators efficiently by
calculating their matrix exponentials locally.

Since in fact $\sigma^x$ is related to $\sigma^z$ by a Hadamard rotation. $T$,
we can compute:
$$
    \exp(\phi \sigma^x)
        = \exp(\phi T \sigma^z T)
        = T \exp(\phi \sigma^z) T
    .
$$
Let's demonstrate:

In [ ]:
hd = np.array([[1, 1], [1, -1]]) * np.sqrt(0.5)

In [ ]:
expm(sx)

In [ ]:
expm(hd @ sz @ hd)

In [ ]:
hd @ expm(sz) @ hd

We might prefer to use the result in the assignment that for a 1-site operator:
$$
    \exp(i t \bm n \cdot \bm \sigma)
        = \cos(t) + i \sin(t) \bm n \cdot \bm \sigma
    .
$$
For imaginary time evolution:
$$
    \exp(\tau \bm n \cdot \bm \sigma)
        = \cos(-i \tau) + i \sin(-i \tau) \bm n \cdot \bm \sigma
        = \cosh(\tau) + \sinh(\tau) \bm n \cdot \bm \sigma
    .
$$

In [ ]:
# Verify formula expontentiation formulas
hx, hz = 1.05, 0.5
hn = np.sqrt(hx ** 2 + hz ** 2)
np.allclose(
    expm((hx * sx + hz * sz) / hn),
    np.cosh(1) * np.eye(2) + np.sinh(1) * ((hx * sx + hz * sz) / hn)
)

## Action

At this stage, I have layed out the Suzuki-Trotter decomposition as well as how
to represent the gates within each term, so we can go ahead and do TEBD.
We are told to evolve a ferromagnetic state:
\begin{align}
    \ket{\psi (t=0)}
        &= \ket{\downarrow} \otimes \cdots \otimes \ket{\downarrow}
    ,
    \\\\
    \ket{\phi (t=0)}
        &= \ket{\uparrow} \otimes \cdots \otimes \ket{\uparrow}
    .
\end{align}

In [ ]:
L  = 12
d  = 2
hx = 1.05
hz = 0.5

In [ ]:
down = np.array([1., 0.]).reshape(2, 1)
up   = down[::-1].reshape((2, 1))

# build wavefunction in MPS representation
ψ = tensor.mps(L=L, d=d)
ψ.from_arr([ down for _ in range(L) ], center=-1)
ϕ = tensor.mps(L=L, d=d)
ϕ.from_arr([ up for _ in range(L) ], center=-1)

We are asked to calculate the energy of this state which requires a Hamiltonian.

In [ ]:
sx = np.diag([1., 1.])[::-1]
sz = np.diag([1.,-1.])

# Build pieces of Hamiltonian in gate representation
def build_pieces_of_H (L, d, hx, hz):
    """Build the field, odd, and even term Hamiltonians and also their union."""
    H_field = np.empty(L, dtype='O')
    for i in range(H_field.size):
        H_field[i] = tensor.mpo(L=L, d=d)
        H_field[i].set_local_oper(-(hx * sx + hz * sz), i + 1)
    H_odd = np.empty(L//2, dtype='O')
    for i in range(H_odd.size):
        H_odd[i] = tensor.mpo(L=L, d=d)
        H_odd[i].set_local_oper(-sz, 2*i + 1)
        H_odd[i].set_local_oper(sz, 2*i + 1 + 1)
    H_even = np.empty(L//2 - 1 + L%2, dtype='O')
    for i in range(H_even.size):
        H_even[i] = tensor.mpo(L=L, d=d)
        H_even[i].set_local_oper(-sz, 2*(i + 1))
        H_even[i].set_local_oper(sz, 2*(i + 1) + 1)
    H_full = np.array([*H_field, *H_odd, *H_even], dtype='O')
    return (H_field, H_odd, H_even, H_full)

In [ ]:
H_field, H_odd, H_even, H_full = build_pieces_of_H(L=L, d=d, hx=hx, hz=hz)

In [ ]:
for name, wave in [('psi', ψ), ('phi', ϕ)]:
    print('MPS Energy of', name, sum(e.expval(wave) for e in H_full))

As a verification, let's compare this to the direct calculation from the Hamiltonian.

In [ ]:
ψ.merge_bonds()
ψ.reset_pos()
psic = ψ[0].mat.reshape(ψ.size())
np.inner(psic, models.tfim_z.H_vec(psic, L, hx, 'o', hz))

In [ ]:
ϕ.merge_bonds()
ϕ.reset_pos()
phic = ϕ[0].mat.reshape(ϕ.size())
np.inner(phic, models.tfim_z.H_vec(phic, L, hx, 'o', hz))

In [ ]:
# Restore MPS compression to these product states
ψ.split_quanta(-1, trim = 1)
ϕ.split_quanta(-1, trim = 1)

### Implementation

Somehow, my code was influenced by the interface to the Julia
[ITensor](https://itensor.org/) package. I haven't used the package, but
seeing that in their code they define everything in terms of indices
and operations on them convinced me I should write classes which do the
same. Keeping track of indices is a pain, so reducing everything to index
operations on my MPS representation (which is basically a list of matrices,
where each matrix has a multiindex for both rows and columns) hopefully
makes algorithms less difficult to program when the essential methods work.

In order to do this, I made many backwards-incompatible changes to my
previous MPS code, which was rather simple to begin with.
Now, there is a TON of stuff going on under the hood to provide a composable
set of elementary tensor operations that are abstracted away from the storage
and representation details of the tensors themselves.
The basic problems are: how do you keep track of a set of indices as a data
structure, and how to you selectively operate on these indices with operations
such as contractions, reshapes, and reorderings.
A lot of this comes down to giving bonds pointers to sites.
When these tasks can, to some extent, be automated, then they open the door
to splitting multi-indices with SVD's, to MPS canonical forms, and to tensor
network contractions.
The icing on the cake, so to speak, is taking these operations and composing
them into algorithms for quantum simulation, such as TEBD, which we are about to
undertake, as well as DMRG (maybe next time?).
The difficulty of these algorithms is in applying gates repeatedly to different
groups of indices, creating a lot of book-keeping.
There are _a lot_ of books to keep.

To be clear, there is no attempt to optimize code, mainly just a framework for
matrix operations where the row and column indices are replaced by multiindices.
It maybe succeeds in reducing code repetition when actually expressing algorithms.
For the programmer, the price of abstraction is payed by usability.
I want to be able to apply an operator on a specific set of indices without
having to manually manipulate the matrices each time I do an experiment.

I also started to 1-index the sites in the tensor trains, which is bound to
create confusion!



### Greatest achievement of this course

The following function is the greatest function I've written:

In [ ]:
def multi_index_perm (dims, perm):
    """Return a slice to permute a multi-index in the computational basis.
    
    `dims` and `perm` enumerate 0 to N-1 as the fastest-changing dit to slowest.
    E.g. perm = np.arange(N) is the identity permutation.

    Arguments:
    dims :: np.ndarray(N) :: dimensions of each index
    perm :: np.ndarray(N) :: perm[i] gives the new position of i
    """
    assert dims.size == perm.size
    iperm = np.argsort(perm)
    new_dims = dims[iperm]
    index = np.zeros(np.prod(dims), dtype='int64')
    for i in range(len(dims)):
        index += np.tile(
            np.repeat(
                np.prod(dims[:iperm[i]], dtype='int64')
                * np.arange(dims[iperm[i]]),
                repeats=np.prod(new_dims[:i], dtype='int64')
            ),
            reps=np.prod(new_dims[i+1:], dtype='int64')
        )
    return index

It performs any arbitrary permutation of a multi-index (with any dimensions).
It's great to have this function because it gives unlimited flexibility in the
manipulation of indices within a multi-index in order to sort indices after 
sophisticated tensor manipulations.
Whoever uses this function has to be highly aware of the issue of endianness,
which I believe asserts itself in both the order of the array `dims` as well as
the order of `np.tile` and `np.repeat`.
This function generalizes `ph121c_lxvm.basis.schmidt.permute`, which can only
be applied to multi-indices where each index has dimension 2, which was useful
earlier for qubit chains, but cannot be generalized because its algorithm is
based on bit swaps, and bit indices always have dimension 2.
The function is great because it works: it is the endgame of indices.

I have no idea if `np.ravel_multi_index` does the same thing: I suspect it 
doesn't. In principle, this function should overcome the limitations imposed
by the maximal `ndim` set by `numpy`, though of course too many dimensions will
lead to dangerously large arrays.

Here is an example of how to create and invert a permutation with both methods:

In [ ]:
tensor.multi_index_perm(
    np.array([2,2,2,2,2,2]),
    np.array([5,2,0,3,1,4])
)[tensor.multi_index_perm(
    np.array([2,2,2,2,2,2]),
    np.array([2,4,1,3,5,0])
)]

In [ ]:
basis.schmidt.permute(
    basis.schmidt.permute(
        np.arange(64), [], 6, perm=np.array([5,2,0,3,1,4])
    ), [], 6, perm=np.array([2,4,1,3,5,0])
)

### Everything else

If you read the source code in `ph121c_lxvm.tensor`, you might think that it is
disgusting, and I would agree with you. The code mainly enforces the topology of
a tensor network and automates operations on the network. Should it be 10 lines
of code? Probably. It's over 1500.

Also, Python does not have a compiler that enforces types, so it is up to the
programmer to enforce type stability. 
For me, that meant writing code in a LBYL (look before you leap) style, whereas
some Python code is better in the EAFP (it’s easier to ask for forgiveness than 
permission) style.

The structure of the `tensor` module is the following: an `index` class is defined
by subclassing `collections.UserList` and adding two attributes and some methods.
Then `multi_index`, `bond`, and `quanta` are subclasses of `index` and they 
enforce some restrictions of the types of elements and attributes in the index.
Separately, the `site` type is a collection of a `np.ndarray` with `ndim=2` and 
a `multi_index` of two `multi_index`es representing the row and column indices. 
The `site` implements SVD, contraction, and sorting operations on the matrix indices. 
Then the `train` type is a `UserList` of `site`s with methods to manipulate these.
Lastly, `mps` and `mpo` are subclasses of `train` with particular methods.

I'll also mention that preserving the topology of the network (i.e. pointers between sites)
during operations which involve not in-place operations such as contractions
requires a bit of wizardry to keep the references altogether.
The process behaves a fair bit like DNA replication.

![replication](https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/Eukaryotic_DNA_replication.svg/800px-Eukaryotic_DNA_replication.svg.pnghttps://upload.wikimedia.org/wikipedia/commons/thumb/1/18/Eukaryotic_DNA_replication.svg/800px-Eukaryotic_DNA_replication.svg.png)

I only implemented some things this way because the `deepcopy` function in the `copy` module apparently knows how to do this correctly.

By the way, the module I wrote is neither complete nor robust.
You cannot yet compose things as I had hoped for, but hopefully
time evolution should work reliably

### Direct's approach

We will apply the gates in the Hamiltonian directly, which necessitates
regrouping the physical indices into sites which match the gates.

We will first exponentiate each group in the Hamiltonian

In [ ]:
# Construct propagators
def build_propagators (L, d, δτ, H_field, H_odd, H_even):
    """Exponentiate each non-commuting piece of the Hamiltonian"""
    U_field = tensor.mpo(L=L, d=d)
    for i, e in enumerate(H_field):
        U_field.set_local_oper(expm(-δτ * e[0].mat), i+1)
    U_odd = tensor.mpo(L=L, d=d)
    for i, e in enumerate(H_odd):
        U_odd.set_local_oper(
            expm(-δτ * np.kron(e[1].mat, e[0].mat)),
            2 * i + 1
        )
    U_even = tensor.mpo(L=L, d=d)
    for i, e in enumerate(H_even):
        U_even.set_local_oper(
            expm(-δτ * np.kron(e[1].mat, e[0].mat)),
            2 * (i + 1)
        )
    return (U_field, U_odd, U_even)

Let's evolve imaginary time

In [ ]:
L  = 12
d  = 2
hx = 1.05
hz = 0.5
δτ = 0.1

In [ ]:
%%time
# States
wave = deepcopy(ψ)
cave = deepcopy(ϕ)
# Max rank
chi = 16
# Operators
H_field, H_odd, H_even, H_full = build_pieces_of_H(L=L, d=d, hx=hx, hz=hz)
U_field, U_odd, U_even = build_propagators(
    L=L, d=d, δτ=δτ, H_field=H_field, H_odd=H_odd, H_even=H_even
)
# Results
ψ_energies = [sum(e.expval(wave) for e in H_full)]
ϕ_energies = [sum(e.expval(cave) for e in H_full)]
# TEBD pattern
Nstp = 10
for _ in range(Nstp):
    for e in [U_field, U_even, U_odd]:
        e.oper(wave, inplace=True)
    wave.canonize(center=-1)
    wave.trim_bonds(chi)
    wave.normalize()
    ψ_energies.append(sum(e.expval(wave) for e in H_full))
for _ in range(Nstp):
    for e in [U_field, U_even, U_odd]:
        e.oper(cave, inplace=True)
    cave.canonize(center=-1)
#     cave.trim_bonds(chi)
    cave.normalize()
    ϕ_energies.append(sum(e.expval(cave) for e in H_full))

In [ ]:
# Hamiltonian system: energy should be conserved
plt.plot(ψ_energies, label='$\psi$')
plt.plot(ϕ_energies, label='$\phi$')
plt.xlabel(f'Number of imaginary time steps at size {δτ}')
plt.ylabel('Expectation value of Hamiltonian')
plt.legend()
plt.show()

Simulation worked! The states are loosing energy and 
We'll come back to this after the next section to do some physics.

### MPO approach

Here I outline the approach I described that was relayed to me by Gil.
There is also an excellent explanation of this on the Tensor Network website.

Start by taking the set of operators, splitting them, stacking them.
Then do TEBD my applying the stacked operator.
Potentially the bond dimension is larger than in the other case,
but analytically it should be the same.

In [ ]:
U_field.split_quanta(-1)
U_odd.split_quanta(-1)
U_even.split_quanta(-1)
U_full = U_field.oper(U_odd.oper(U_even))

Well, debugging is hard, and that looks like a bond reference failed to update.
This means my `mpo.oper` method that I use on `mps` types doesn't generalize
to `mpo` types. I thought I should write a generic function on `train` types, but 
it doesn't seem necessary.

## Convergence to ground state

We will study the convergence to the ground state of the MPS wavefunction of $\ket{\psi}$ for system sizes $L=12, 32, 64, 128$.
For larger systems, we will 

### Comparing to ED

Let's setup a quick calculation for the ground state energy at the small
system using a spare eigensolver and our exact methods from earlier in the course.

In [ ]:
%time Eₒ = eigsh(models.tfim_z.H_oper(L, hx, 'o', hz), k=1, return_eigenvectors=False)[0]

In [ ]:
Eₒ

Well, that was quick (thank you Fortran) and seems really close to the
minimum reached by our earlier time evolution.
We'll do the same run this time except by truncating the simulation
when it converges to within 0.1% of its previous value

### Larger system sizes

Let's boost to a larger systems and compute their approximate ground states.
We can measure correlation functions in these ground states and
compare their energies.
We will not be measuring their energies at each time step, but only every
10 time steps to reduce the computational cost

## a Néel-ing

Let's see the convergence to the ground state of an alternating spin chain.

\begin{align}
    \ket{\psi (t=0)}
        &= \ket{\uparrow} \otimes \ket{\downarrow} \otimes \cdots
    .
\end{align}

In [ ]:
assert (L % 2 == 0)
ρ = tensor.mps(L=L, d=d)
wave = []
for i in range(L // 2):
    wave.append(up)
    wave.append(down)
ρ.from_arr(wave, center=-1)    

## Extra reading

You might also be interested in reading
- [Feynman on simulation](https://link.springer.com/article/10.1007/BF02650179)
- [Quantum computation by a Caltech sophomore](https://arxiv.org/abs/2103.12783)